In [1]:
import timm
from torch import nn
from timm.data.loader import create_loader
import torch.utils.data as data

import pandas as pd



In [2]:

class MultiLabelModel(nn.Module):
    def __init__(self, model, n_classes):
        super().__init__()
        self.base_model = model
#         self.out = nn.Sequential(
#             nn.Dropout(p=0.2),
#             nn.Linear(in_features=last_channel, out_features=n_classes)
#         )
        self.WEIGHTS = [1/n_classes]*n_classes
        self.num_classes = n_classes

    def forward(self, x):
        x = self.base_model(x)
        
        x = torch.flatten(x, 1)

        return x

    def get_loss(self,loss_fn, output, target):
        losses = 0.0
        for i in range(0,self.num_classes):
            losses+=self.WEIGHTS[i]* loss_fn(output[i], target[i].cuda())
        return losses
    def as_sequential_for_ML(self):

        layers = [self.conv_stem, self.bn1, self.act1]
        layers.extend(self.blocks)
        layers.extend([self.conv_head, self.bn2, self.act2])
        return nn.Sequential(*layers)
    
from PIL import Image


In [3]:
class DatasetMultiLabel(data.Dataset):
    def __init__(
            self,
            annotation_path,
            transform=None):

        super().__init__()

        self.transform = transform

        # initialize the arrays to store the ground truth labels and paths to the images
        self.data = []
        self.labels = []
        self.label_names = []
        # read the annotations from the CSV file
        df = pd.read_csv(annotation_path)
        cols = df.columns
        self.label_names = list(cols[1:])
        for i,row in df.iterrows():
            lb = []
            for j in cols:
                if j=='image_path':
                    self.data.append(row[j])
                else:
                    lb.append(float(row[j]))
            self.labels.append(lb)
                
    def __getitem__(self, idx):
        # take the data sample by its index
        img_path = self.data[idx]

        # read image
        img = Image.open(img_path)

        # apply the image augmentations if needed
        if self.transform:
            img = self.transform(img)

        labels = self.labels[idx]

        return img, labels

    def __len__(self):
        # return len(self.samples)
        return len(self.data)

In [5]:
dataset_train = DatasetMultiLabel("img_align_celeba/train.csv")
dataset_val = DatasetMultiLabel("img_align_celeba/val.csv")

In [6]:
from timm.data import (
    resolve_data_config,
)
from timm.models import create_model

In [56]:
dataset_train.label_names

['Eyeglasses',
 'Wearing_Earrings',
 'Wearing_Hat',
 'Wearing_Necklace',
 'Wearing_Necktie']

In [7]:
model = create_model(
    'efficientnet_b0',
    num_classes=len(dataset_val.labels[0]),
)

In [8]:
model = MultiLabelModel(
        model,
        n_classes=len(dataset_val.labels[0]),
    ).cuda()

In [9]:
from timm.optim import create_optimizer
import pickle

In [10]:
args = pickle.load(open("PyTorch-Image-Models-Multi-Label-Classification/args.p",'rb'))

In [11]:
from types import SimpleNamespace
args = SimpleNamespace()
args.weight_decay = 0
args.lr = 1e-4
args.opt = 'adam' #'lookahead_adam' to use `lookahead`
args.momentum = 0.9


In [12]:
optimizer = create_optimizer(args, model)

In [13]:
data_config = resolve_data_config(
       args = {}, model=model
    )

In [14]:
loader_train = create_loader(
    dataset_train,
    input_size=data_config["input_size"],
    batch_size=64,
    is_training=True,
    mean=data_config["mean"],
    std=data_config["std"],
   interpolation=data_config["interpolation"],
)

In [15]:
from timm.scheduler import create_scheduler


In [16]:
args.epochs = 1000

In [17]:
args.sched = "step"
args.decay_epochs = 30
args.decay_rate = 0.1
args.warmup_lr = 0.0001
args.min_lr = 1e-5
args.warmup_epochs = 3

In [18]:
lr_scheduler, num_epochs = create_scheduler( args,optimizer)

In [19]:
loader_val = create_loader(
    dataset_val,
    input_size=data_config["input_size"],
    batch_size=64,
    is_training=False,
    mean=data_config["mean"],
    std=data_config["std"],
    interpolation=data_config["interpolation"],
   
)

In [20]:
import torch    

In [21]:
model.eval();

In [22]:
loss_fn = nn.BCELoss()


In [58]:
def validate(loader,model):
    with torch.no_grad():
        total_loss = 0
        m = nn.Sigmoid()
        ops = []
        labels = []
        preds = []
        for batch_idx, (input, target) in enumerate(loader):
            input = input.cuda()
            labels.append(target.detach().cpu())
            target = target.float().cuda()
            output = m(model(input))
            loss = model.get_loss(loss_fn, output, target)
            total_loss += loss.item()
            pred_model = (output>0.5).detach().cpu()
            preds.append(pred_model)


        num_of_batches_per_epoch = len(loader)
        avg_loss = total_loss / num_of_batches_per_epoch
        print("AVERAGE LOSS:",avg_loss)
        preds = torch.cat(preds).int()
        labels = torch.cat(labels).int()
        acc_score = accuracy_score(labels,preds)
        print("MULTILABEL accuracy score:",acc_score)
        per_class = []
        for i in range(len(preds.T)):
            per_class.append(accuracy_score(labels.T[i],preds.T[i]))
        print(dataset_train.label_names)
        print(per_class)
        return avg_loss

In [60]:
validate(loader_val,model)

AVERAGE LOSS: 0.6931395322987528
MULTILABEL accuracy score: 0.07576840600428877
['Eyeglasses', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Necklace', 'Wearing_Necktie']
[0.7203955206099595, 0.45604002859185133, 0.6878722897307601, 0.4949964260185847, 0.4089826066237789]


0.6931395322987528

In [26]:
from sklearn.metrics import balanced_accuracy_score


In [32]:
from imp import reload


In [33]:
reload(sklearn)

<module 'sklearn' from '/usr/local/lib/python3.8/dist-packages/sklearn/__init__.py'>

In [34]:
import sklearn

In [35]:
sklearn.__version__

'1.1.2'

In [40]:
import numpy as np
from sklearn.metrics import accuracy_score

In [51]:
len(preds.T)

5

In [49]:
accuracy_score(labels,preds)

0.07576840600428877

In [77]:
def train_one_epoch(
    num_of_data_train,
    epoch,
    model,
    loader,
    optimizer,
    loss_fn,
    lr_scheduler=None,
    saver=None,
    output_dir="",
):
    second_order = hasattr(optimizer, "is_second_order") and optimizer.is_second_order
    model.train()
    total_loss = 0
    m = nn.Sigmoid()
    ops = []
    labels = []
    preds = []
    ct=0
    for batch_idx, (input, target) in enumerate(loader):
            input = input.cuda()
            ct+=1
            labels.append(target.detach().cpu())
            target = target.float().cuda()
            output = m(model(input))
            loss = model.get_loss(loss_fn, output, target)
            total_loss += loss.item()
            pred_model = (output>0.5).detach().cpu()
            preds.append(pred_model)
            optimizer.zero_grad()
            loss.backward(create_graph=second_order)
            optimizer.step()
            if ct%10==0:
                print("LOSS:",loss.item())
    num_of_batches_per_epoch = len(loader)
    avg_loss = total_loss / num_of_batches_per_epoch
    print("AVERAGE LOSS:",avg_loss)
    preds = torch.cat(preds).int()
    labels = torch.cat(labels).int()
    acc_score = accuracy_score(labels,preds)
    print("MULTILABEL accuracy score:",acc_score)
    per_class = []
    for i in range(len(preds.T)):
        per_class.append(accuracy_score(labels.T[i],preds.T[i]))
    print(dataset_train.label_names)
    print(per_class)
    return avg_loss

In [64]:
from timm.utils import CheckpointSaver


In [68]:
saver = CheckpointSaver(
            model=model,
            optimizer=optimizer,
    checkpoint_dir="weights"
        )

In [67]:
eval_metrics = validate(loader_val,model)

AVERAGE LOSS: 0.6931395322987528
MULTILABEL accuracy score: 0.07576840600428877
['Eyeglasses', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Necklace', 'Wearing_Necktie']
[0.7203955206099595, 0.45604002859185133, 0.6878722897307601, 0.4949964260185847, 0.4089826066237789]


In [72]:
!mkdir weights

In [74]:
save_metric = eval_metrics
best_metric, best_epoch = saver.save_checkpoint(
                    1, metric=save_metric
                )

In [75]:
num_of_data_train = len(dataset_train)

In [ ]:
for epoch in range(0,1000):
    loss_train = train_one_epoch(
    num_of_data_train,
    epoch,
    model,
    loader_train,
    optimizer,
    loss_fn,
    lr_scheduler=lr_scheduler,
)
    saver.save_checkpoint(epoch,metric=loss_train)
    validate(loader_val,model)

LOSS: 0.531355619430542
LOSS: 0.5261980295181274
LOSS: 0.48028263449668884
LOSS: 0.5054075121879578
LOSS: 0.6295892596244812
LOSS: 0.5074624419212341
LOSS: 0.4384707808494568
LOSS: 0.9877334833145142
LOSS: 0.38097047805786133
LOSS: 0.4639021158218384
LOSS: 0.4766576588153839
LOSS: 0.4930850565433502
LOSS: 0.4200809597969055
LOSS: 0.5419005155563354
LOSS: 0.45323246717453003
LOSS: 0.5344032645225525
LOSS: 0.5881763696670532
LOSS: 0.4452283978462219
LOSS: 0.5987139344215393
LOSS: 0.560493528842926
LOSS: 0.4783082902431488
LOSS: 0.4484843313694
LOSS: 0.46433520317077637
LOSS: 0.7476933002471924
LOSS: 0.6828913688659668
LOSS: 1.27458655834198
LOSS: 1.2565944194793701
LOSS: 0.4073219895362854
LOSS: 0.5153696537017822
LOSS: 0.46481823921203613
LOSS: 0.636134147644043
LOSS: 0.9804605841636658
LOSS: 0.6530095934867859
LOSS: 0.46081069111824036
LOSS: 0.4393851161003113
LOSS: 0.44275766611099243
LOSS: 0.5818337202072144
LOSS: 0.5245600938796997
LOSS: 0.4602753818035126
LOSS: 0.6111949682235718
L

In [80]:
validate(loader_val,model)

AVERAGE LOSS: 1.319113413944389
MULTILABEL accuracy score: 0.0736240171551108
['Eyeglasses', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Necklace', 'Wearing_Necktie']
[0.8314272099118418, 0.5141767929473433, 0.8825351441505838, 0.6566595187038361, 0.8195139385275196]


1.319113413944389